### Factors Driving Life Expectancy : Do immunization matters?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
#MICE performe multiple regression for imputing / it is a very robost model for imputation
from fancyimpute import IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest,  f_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
import xgboost as xgb

import os

from library.sb_utils import save_file

In [2]:
sns.set()
sns.set_context("notebook")

### Import dataset 

In [3]:
df1 = pd.read_csv("../data/life_exp_clean.csv")

In [4]:
# convert Year to datetime object 
df1.Year = pd.to_datetime(df1.Year, format='%Y-%d-%m')
df1['year']=df1['Year'].dt.year

### Label Encoder categerical variable

In [214]:
# copy the data
df = df1.copy(deep=True)

In [215]:
df = df.drop(columns='Country')

In [216]:
# one hot_encoder
df = pd.get_dummies(df)
df.insert(0,'Country', df1['Country'], allow_duplicates=True)

In [217]:
df.columns

Index(['Country', 'Year', 'Life_expectancy', 'Adult_Mortality',
       'infant_deaths', 'Alcohol', 'percentage_expenditure', 'Hepatitis_B',
       'Measles', 'BMI', 'under_five_deaths', 'Polio', 'Total_expenditure',
       'Diphtheria', 'HIV/AIDS', 'GDP', 'Population', 'thinness__1_19_years',
       'thinness_5_9_years', 'Income_composition_of_resources', 'Schooling',
       'year', 'population', 'Status_Developed', 'Status_Developing'],
      dtype='object')

In [218]:
columns = ['Country', 'Year', 'Adult_Mortality',
       'infant_deaths', 'Alcohol', 'percentage_expenditure', 'Hepatitis_B',
       'Measles', 'BMI', 'under_five_deaths', 'Polio', 'Total_expenditure',
       'Diphtheria', 'HIV/AIDS', 'GDP', 'Population', 'thinness__1_19_years',
       'thinness_5_9_years', 'Income_composition_of_resources', 'Schooling',
       'year', 'Status_Developed', 'Status_Developing']
# define X and y
X = df[columns]
y = df[['Life_expectancy']]

In [219]:
# split the dataset     
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=47)

In [220]:
# save name
names_list = ['Country','Year', 'year']
names_train = X_train[names_list]
names_test = X_test[names_list]
X_train.drop(columns=names_list, inplace=True)
X_test.drop(columns=names_list, inplace=True)
X_train.shape, X_test.shape

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


((2342, 20), (586, 20))

In [221]:
# traitement of missing data with MICE Imputation
mice_imputer = IterativeImputer()
X_train_imput = mice_imputer.fit_transform(X_train)
X_test_imput = mice_imputer.fit_transform(X_test)

In [222]:
# scale the date
scaler = StandardScaler()
X_train_imput_scale = scaler.fit_transform(X_train_imput)
X_test_imput_scale = scaler.fit_transform(X_test_imput)

In [223]:
# dimentionnality reduction
pca = PCA()
X_train_pca = pca.fit(X_train_imput_scale)

In [224]:
X_train_pcat = pca.transform(X_train_imput_scale)

###  Modeling


##### Function to evaluate the model

In [ ]:
def model_eval(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')

### 1. Model linear_regression 

In [ ]:
pipe = make_pipeline(
    IterativeImputer(), 
    StandardScaler(),
    SelectKBest(f_regression),
    LinearRegression()
)

In [ ]:
# define the k range to investigate
k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}

In [ ]:
reg_grid_cv = GridSearchCV(pipe, param_grid=grid_params, cv=10, n_jobs=-1)

In [ ]:
reg_grid_cv.fit(X_train, y_train)

In [ ]:
#Print the `best_params_` attribute of 'red_grid_cv`
reg_grid_cv.best_params_

In [ ]:
score_mean = reg_grid_cv.cv_results_['mean_test_score']
score_std = reg_grid_cv.cv_results_['std_test_score']
cv_k = [k for k in reg_grid_cv.cv_results_['param_selectkbest__k']]
best_k = reg_grid_cv.best_params_['selectkbest__k']

In [ ]:
best_k = reg_grid_cv.best_params_['selectkbest__k']
plt.subplots(figsize=(10, 5))
plt.errorbar(cv_k, score_mean, yerr=score_std)
plt.axvline(x=best_k, c='r', ls='--', alpha=.5)
plt.xlabel('k')
plt.ylabel('CV score (r-squared)')
plt.title('Pipeline mean CV score (error bars +/- 1sd)');

The above suggests a good value for k is 19

In [ ]:
# mask to select the most useful features 
selected_features= reg_grid_cv.best_estimator_.named_steps.selectkbest.get_support()
selected_features

In [ ]:
coefs = reg_grid_cv.best_estimator_.named_steps.linearregression.coef_[0]
features = X_train.columns[selected_features]
pd.Series(coefs, index=features).sort_values(ascending=False)

#### Note:
From this result, the best positive features here are Income_composition_of_resources , followed by infant_deaths. 

Features which negatively influence life expectancy are under_five_deaths of children (the most important) , next it is HIV/AIDS and Adult_Mortality.

We realise that immunization factors are also present among the positive features (come in 6th , 7th and 8th positions) even if their strength is not high as compared to the first features. 

It is also important to highlight the fact that the feature: status of the country (either developed or developing) has an impact on the model. 
May, it would be great to expect if a dataset with only developing countries would highlight the importance of immunization as we explore in the EDA.


In [ ]:
reg_grid_cv.best_estimator_

In [ ]:
lin_reg = reg_grid_cv.best_estimator_

In [ ]:
#lin_reg.get_params()

In [ ]:

#### a suprimer 
pipe2 = make_pipeline(
    IterativeImputer(), 
    StandardScaler(),
    SelectKBest(f_regression, k=19),
    LinearRegression())

#### Assess performance on train and test data

In [ ]:
predicted_train = lin_reg.predict(X_train)
predicted_test = lin_reg.predict(X_test)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

#### Assessing performance using cross-validation

In [ ]:
cv_results = cross_validate(reg_grid_cv.best_estimator_, X_train, y_train, cv=5)

In [ ]:
cv_scores = cv_results['test_score']
np.mean(cv_scores), np.std(cv_scores)

### 2. Model linear regression with PCA

In [ ]:
pipe3 = make_pipeline(
    IterativeImputer(), 
    StandardScaler(),
    PCA(n_components=19),
    LinearRegression())

In [34]:
lin_reg2 = pipe3.fit(X_train, y_train)

In [35]:
#lin_reg2.get_params()

#### Model Evaluation

In [36]:
predicted_train = lin_reg2.predict(X_train)
predicted_test = lin_reg2.predict(X_test)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 2.4646197667876497
MSE: 11.782041384887318
RMSE: 3.432497834651512
R2 Square 0.8682964325352108
__________________________________
MAE: 2.42219185750846
MSE: 10.749188467196356
RMSE: 3.2785955022229194
R2 Square 0.881789518940812
__________________________________


### 3. Model: Ridge regression 

In [37]:
###select the best value of alpha

In [38]:
# Perform 5-fold cross-validation: ridge_cv
alpha = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 20.0, 40.0, 60.0, 80.0 ,100.0]
for x in alpha:
    ridge = Ridge(alpha=10)
    ridge_cv = cross_val_score(ridge, X_train_imput_scale, y_train, cv=10, n_jobs=-1)
    print( 'cross-validated scores of ridge regression for alpha', x ,'is :', np.mean(ridge_cv))

cross-validated scores of ridge regression for alpha 1e-05 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 0.0001 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 0.001 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 0.01 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 0.1 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 0.0 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 1.0 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 10.0 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 20.0 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 40.0 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 60.0 is : 0.8783486661081282
cross-validated scores of ridge regression for alpha 80.0 is : 0.8783486661081282
cross-validated

##### use of GridSearchCV (it doesn't work ????)

In [39]:
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 20.0, 40.0, 60.0, 80.0 ,100.0]
grid

{'alpha': [1e-05,
  0.0001,
  0.001,
  0.01,
  0.1,
  0.0,
  1.0,
  10.0,
  20.0,
  40.0,
  60.0,
  80.0,
  100.0]}

In [40]:
pipe4 = make_pipeline(
    IterativeImputer(), 
    StandardScaler(),
    Ridge())

In [41]:
ridge_grid_cv = GridSearchCV(pipe4, grid, cv=10, n_jobs=-1)

In [42]:
# not working , why?????
#ridge_grid_cv.fit(X_train, y_train)

In [43]:
#ridge_grid_cv.best_params_

#####  Model: Ridge regression 

In [44]:
pipe5 = make_pipeline(
    IterativeImputer(), 
    StandardScaler(),
    Ridge(alpha=0.5))

In [45]:
ridge_reg = pipe5.fit(X_train, y_train)

In [46]:
predicted_train = ridge_reg.predict(X_train)
predicted_test = ridge_reg.predict(X_test)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 2.4665076458596196
MSE: 11.789201581196023
RMSE: 3.433540677084811
R2 Square 0.8682163934853723
__________________________________
MAE: 2.4230317672145
MSE: 10.750125309298998
RMSE: 3.278738371584259
R2 Square 0.881779216343926
__________________________________


In [47]:
#ridge_reg.get_params()

### 4.ElasticNet regression

In [48]:
#  hyperparameter grid
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = np.arange(0, 1.1, 0.1)

In [49]:
grid

{'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0.0, 1.0, 10.0, 100.0],
 'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])}

###### identify the best parameters

In [50]:
elastic_net = ElasticNet()

EL_grid_cv0 = GridSearchCV(elastic_net, grid, cv=5, n_jobs=-1)

EL_grid_cv0 .fit(X_train_imput_scale, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5057.171233473611, tolerance: 21.296419035439794
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0.0, 1.0,
                                   10.0, 100.0],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])})

In [51]:
print("Tuned ElasticNet l1 ratio: {}".format(EL_grid_cv0.best_params_))

Tuned ElasticNet l1 ratio: {'alpha': 0.0001, 'l1_ratio': 0.4}


##### Run the model with the best paramters and assessing the performance

In [52]:
elastic_net = ElasticNet(alpha = 0.0001, l1_ratio = 0.4)

elastic_net_CV = cross_val_score(elastic_net, X_train_imput_scale, y_train, cv=10, n_jobs=-1)

print( 'cross-validation scores of elastic_net regression is ', np.mean(elastic_net_CV))

cross-validation scores of elastic_net regression is  0.8794156394145368


In [53]:
Elastic_net = elastic_net.fit( X_train_imput_scale, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5057.171233473611, tolerance: 21.296419035439794
  model = cd_fast.enet_coordinate_descent(


In [54]:
predicted_train = Elastic_net.predict(X_train_imput_scale)
predicted_test = Elastic_net.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 2.527914693963443
MSE: 12.300043713472308
RMSE: 3.5071418154206864
R2 Square 0.8625060306514405
__________________________________
MAE: 2.422465123600006
MSE: 10.74930620707784
RMSE: 3.2786134580151165
R2 Square 0.8817882241372021
__________________________________


### 5.  Decision tree for regression

In [55]:
param_rf={'max_depth':[4,6,8, 10, 12, 14, 16], 'min_samples_leaf': [0.1, 0.2, 0.5, 1, 1.5, 2], 
          'max_features': [0.2,0.4, 0.6, 0.8, 1.0]}

In [56]:
dt = DecisionTreeRegressor(random_state=1)
dt_grid_CV= GridSearchCV(estimator= dt ,param_grid=param_rf, cv=10,
                          scoring='accuracy',  n_jobs=-1)
# Fit dt to the training set
dt_grid_CV=dt_grid_CV.fit(X_train_imput_scale, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warni

In [57]:
best_dt_param = dt_grid_CV.best_params_
best_dt_param

{'max_depth': 4, 'max_features': 0.2, 'min_samples_leaf': 0.1}

In [58]:
model_dt = dt_grid_CV.best_estimator_

In [59]:
predicted_train = dt_grid_CV.predict(X_train_imput_scale)
predicted_test = dt_grid_CV.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 3.8712166019486913
MSE: 28.4318375579228
RMSE: 5.33215130673566
R2 Square 0.682179487099669
__________________________________
MAE: 3.553711820842639
MSE: 23.35047910088829
RMSE: 4.832233345037085
R2 Square 0.7432111851139154
__________________________________


In [60]:
#### experimentation, Use of X_train and X_test (just the missing data imputation, no_scaling)

In [61]:
#model_dt.fit(X_train_imput, y_train)

In [62]:
#predicted_train = model_dt.predict(X_train_imput_scale)
#predicted_test = model_dt.predict(X_test_imput_scale)
#model_eval(y_test, predicted_test)
#model_eval(y_train, predicted_train)

### 6.Random Forest Model

https://medium.datadriveninvestor.com/random-forest-regression-9871bc9a25eb

#### Simple model of random forest 

In [63]:
rf_reg1 = RandomForestRegressor(n_estimators = 200, max_depth= 4, max_features=0.4, n_jobs=-1, random_state=1)
rf_reg1.fit(X_train_imput_scale, y_train)

C:\Users\HP\AppData\Local\Temp/ipykernel_22264/1361632199.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_reg1.fit(X_train_imput_scale, y_train)


RandomForestRegressor(max_depth=4, max_features=0.4, n_estimators=200,
                      n_jobs=-1, random_state=1)

In [64]:
#### evaluate the model

In [65]:
predicted_train = rf_reg1.predict(X_train_imput_scale)
predicted_test = rf_reg1.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 2.101229272755366
MSE: 8.956320005695064
RMSE: 2.992711146384673
R2 Square 0.8998832835862101
__________________________________
MAE: 1.955380700864789
MSE: 7.327613677992686
RMSE: 2.7069565341897692
R2 Square 0.9194171038553456
__________________________________


#### 2nd model of random forest 

In [66]:
rf_reg2 = RandomForestRegressor(n_estimators = 200, max_depth= 10, max_features=0.4, n_jobs=-1, random_state=1)
rf_reg2.fit(X_train_imput_scale, y_train)

C:\Users\HP\AppData\Local\Temp/ipykernel_22264/3700214051.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_reg2.fit(X_train_imput_scale, y_train)


RandomForestRegressor(max_depth=10, max_features=0.4, n_estimators=200,
                      n_jobs=-1, random_state=1)

In [67]:
# evaluate the model
predicted_train = rf_reg2.predict(X_train_imput_scale)
predicted_test = rf_reg2.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.4623943547759204
MSE: 4.929916106159912
RMSE: 2.2203414390944274
R2 Square 0.9448917621935858
__________________________________
MAE: 0.777499479811991
MSE: 1.2670101926638018
RMSE: 1.1256154728253347
R2 Square 0.9860664937786929
__________________________________


In [68]:
##### random forest model

In [69]:
rf_reg3 = RandomForestRegressor(n_estimators = 100, max_depth=7, max_features=0.3, n_jobs=-1, random_state=1)
rf_reg3.fit(X_train_imput_scale, y_train)

C:\Users\HP\AppData\Local\Temp/ipykernel_22264/698106051.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_reg3.fit(X_train_imput_scale, y_train)


RandomForestRegressor(max_depth=7, max_features=0.3, n_jobs=-1, random_state=1)

In [70]:
# evaluate the model
predicted_train = rf_reg3.predict(X_train_imput_scale)
predicted_test = rf_reg3.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.6981721871584778
MSE: 6.205929568166909
RMSE: 2.4911703209870875
R2 Square 0.9306280603791465
__________________________________
MAE: 1.331830701811352
MSE: 3.4361636471429535
RMSE: 1.8536891991763218
R2 Square 0.9622119791678742
__________________________________


#### 4th model of random forest with GridSearchCV

In [71]:
param_rf={'max_depth':[4, 5, 6, 7, 8],'n_estimators':[50, 75, 100, 150, 175, 200], 
          'min_samples_leaf': [0.1, 0.2, 0.5, 0.7, 0.9, 1], 'max_features': [0.3, 0.4, 0.5, 0.6]},

In [72]:
# Perform Grid-Search
RF_grid_CV= GridSearchCV(estimator= RandomForestRegressor(random_state=1) ,param_grid=param_rf, cv=5, 
                          scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

In [73]:
RF_grid_CV.fit(X_train_imput_scale, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-16.46567681 -16.14806641 -16.24551544 -16.23008279 -16.36374627
 -16.21500979 -25.92151165 -25.75779425 -26.08407651 -25.80385491
 -25.95426132 -25.8295139  -90.96085157 -90.9647589  -90.96819255
 -90.97108215 -90.96998753 -90.97150866          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
  -8.76778695  -8.65405795  -8.68811647  -8.71038695  -8.6967627
  -8.64936681 -15.35127237 -15.2068676  -15.27169714 -15.25163269
 -15.25907701 -15.17265097 -24.8954902  -24.48201041 -24.62413834
 -24.5550114  -24.46547393 -24.39902257 -90.96085157 -90.9647589
 -90.96819255 -90.97108215 -90.96998753 -90.97150866          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
    

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid=({'max_depth': [4, 5, 6, 7, 8],
                          'max_features': [0.3, 0.4, 0.5, 0.6],
                          'min_samples_leaf': [0.1, 0.2, 0.5, 0.7, 0.9, 1],
                          'n_estimators': [50, 75, 100, 150, 175, 200]},),
             scoring='neg_mean_squared_error', verbose=1)

In [74]:
best_rf_param = RF_grid_CV.best_params_
best_rf_param

{'max_depth': 8,
 'max_features': 0.6,
 'min_samples_leaf': 1,
 'n_estimators': 200}

In [75]:
rf_reg4 = RF_grid_CV.best_estimator_

In [76]:
predicted_train = rf_reg4.predict(X_train_imput_scale)
predicted_test = rf_reg4.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.5166324754404246
MSE: 5.2092688606991455
RMSE: 2.2823822775116236
R2 Square 0.941769064423985
__________________________________
MAE: 1.0274587227469514
MSE: 2.1669098126828152
RMSE: 1.4720427346659524
R2 Square 0.9761701590635594
__________________________________


In [77]:
Important_features =  pd.Series(rf_reg4.feature_importances_, index = X_train.columns).sort_values()
Important_features.plot(kind='barh', figsize=(10,7) , fontsize=10)
plt.title('Random Forest regressor Features of importance')
plt.show()

In [78]:
# Perform K-Fold CV for Random forest regressor 
scores = cross_val_score(rf_reg4, X_train_imput_scale, y_train, cv= 5)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed whe

In [79]:
np.mean(scores)

0.9502484877866484

In [80]:
scores2 = cross_validate(rf_reg4, X_train_imput_scale, y_train, cv=5)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed whe

In [81]:
scores2 = cv_results['test_score']
np.mean(scores2), np.std(scores2)

(0.8796441263228804, 0.012124405235131887)

In [82]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1500, num = 10)]
max_features =  [x for x in np.linspace(start = 0.2, stop = 1.0, num = 5)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [1, 2, 4, 6]
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 344, 488, 633, 777, 922, 1066, 1211, 1355, 1500], 'max_features': [0.2, 0.4, 0.6000000000000001, 0.8, 1.0], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110], 'min_samples_split': [2, 5, 7, 10], 'min_samples_leaf': [1, 2, 4, 6], 'bootstrap': [True, False]}


In [83]:
# Use the random grid to search for best hyperparameters
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train_imput_scale, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': [0.2, 0.4,
                                                         0.6000000000000001,
                                                         0.8, 1.0],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 7, 10],
                                        'n_estimators': [200, 344, 488, 633,
                                                         777, 922, 1066, 1211,
                                                         1355, 1500]},
                   random_state=42, verbose=2)

In [84]:
rf_random.best_params_

{'n_estimators': 633,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 0.4,
 'max_depth': 40,
 'bootstrap': False}

In [85]:
rf_reg6 = rf_random.best_estimator_

In [86]:
predicted_train = rf_reg6.predict(X_train_imput_scale)
predicted_test = rf_reg6.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.3302398244450553
MSE: 4.289323798076919
RMSE: 2.0710682746053832
R2 Square 0.9520525155392032
__________________________________
MAE: 4.3029400139787997e-13
MSE: 3.0055249644548697e-25
RMSE: 5.482266834489972e-13
R2 Square 1.0
__________________________________


In [87]:
# Perform K-Fold CV
rf_scores = cross_val_score(rf_random.best_estimator_, X_train_imput_scale, y_train, cv=5)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed whe

In [88]:
 np.mean(rf_scores), np.std(rf_scores)

(0.961487494245803, 0.005495752176990227)

##### 1st Gradien Boosting Model 

In [89]:
grdB = GradientBoostingRegressor(n_estimators = 150, max_depth = 3, random_state=1)
grdB.fit(X_train_imput_scale, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(n_estimators=150, random_state=1)

In [90]:
predicted_train = grdB.predict(X_train_imput_scale)
predicted_test = grdB.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.7741461770760325
MSE: 6.3917435035429655
RMSE: 2.5281897681034478
R2 Square 0.9285509705630227
__________________________________
MAE: 1.1864815531020003
MSE: 2.7504461713223574
RMSE: 1.658446915436957
R2 Square 0.9697529198570076
__________________________________


#### 2nd Gradient Boost Model

In [91]:
n_estimators = [int(x) for x in np.linspace(start = 25, stop = 150, num = 5)]
max_features =  [x for x in np.linspace(start = 0.2, stop = 1.0, num = 5)]
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [6, 8, 10, 12]
min_samples_leaf = [4, 6, 8, 10]
learning_rate = [x for x in np.linspace(0.01, 0.3, num = 5)]
criterion = ['mse',]


grd_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'learning_rate': learning_rate}
print(grd_grid)

{'n_estimators': [25, 56, 87, 118, 150], 'max_features': [0.2, 0.4, 0.6000000000000001, 0.8, 1.0], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [6, 8, 10, 12], 'min_samples_leaf': [4, 6, 8, 10], 'learning_rate': [0.01, 0.08249999999999999, 0.155, 0.22749999999999998, 0.3]}


In [92]:
# with RandomizedSearchCV
grd = GradientBoostingRegressor()

grd_Boost = RandomizedSearchCV(estimator = grd, param_distributions= grd_grid,  cv = 5, verbose=1, random_state=1, n_jobs = -1)

In [93]:
# with GridSearchCV
grd = GradientBoostingRegressor(random_state = 1)

grd_Boost2 = GridSearchCV(estimator = grd, param_grid= grd_grid, scoring='neg_mean_squared_error', cv = 5, verbose=1, n_jobs = -1)

In [94]:
grd_Boost.fit(X_train_imput_scale, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.01,
                                                          0.08249999999999999,
                                                          0.155,
                                                          0.22749999999999998,
                                                          0.3],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                                        'max_features': [0.2, 0.4,
                                                         0.6000000000000001,
                                                         0.8, 1.0],
                                        'min_samples_leaf': [4, 6, 8, 10],
                                        'min_samples_split': [6, 8, 10, 12],
                                        'n_estimators': [25, 56, 87, 1

In [95]:
grd_Boost2.fit(X_train_imput_scale, y_train)

Fitting 5 folds for each of 20000 candidates, totalling 100000 fits


C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=1),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.08249999999999999, 0.155,
                                           0.22749999999999998, 0.3],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_features': [0.2, 0.4, 0.6000000000000001, 0.8,
                                          1.0],
                         'min_samples_leaf': [4, 6, 8, 10],
                         'min_samples_split': [6, 8, 10, 12],
                         'n_estimators': [25, 56, 87, 118, 150]},
             scoring='neg_mean_squared_error', verbose=1)

In [96]:
grd_Boost.best_params_

{'n_estimators': 118,
 'min_samples_split': 10,
 'min_samples_leaf': 8,
 'max_features': 0.6000000000000001,
 'max_depth': 10,
 'learning_rate': 0.08249999999999999}

#grd_Boost.best_params_ = {'n_estimators': 118,
 'min_samples_split': 10,
 'min_samples_leaf': 8,
 'max_features': 0.6000000000000001,
 'max_depth': 10,
 'learning_rate': 0.08249999999999999}

In [97]:
grd_Boost2.best_params_

{'learning_rate': 0.08249999999999999,
 'max_depth': 7,
 'max_features': 0.8,
 'min_samples_leaf': 4,
 'min_samples_split': 12,
 'n_estimators': 150}

#grd_Boost2.best_params_ = {'learning_rate': 0.08249999999999999,
 'max_depth': 7,
 'max_features': 0.8,
 'min_samples_leaf': 4,
 'min_samples_split': 12,
 'n_estimators': 150}

In [98]:
grd_Boost.best_estimator_

GradientBoostingRegressor(learning_rate=0.08249999999999999, max_depth=10,
                          max_features=0.6000000000000001, min_samples_leaf=8,
                          min_samples_split=10, n_estimators=118)

In [99]:
grd_Boost2.best_estimator_

GradientBoostingRegressor(learning_rate=0.08249999999999999, max_depth=7,
                          max_features=0.8, min_samples_leaf=4,
                          min_samples_split=12, n_estimators=150,
                          random_state=1)

In [100]:
GRD_Boost= grd_Boost.best_estimator_

In [101]:
GRD_Boost2= grd_Boost2.best_estimator_

In [102]:
predicted_train = GRD_Boost.predict(X_train_imput_scale)
predicted_test = GRD_Boost.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.430898916922584
MSE: 4.4758386975356395
RMSE: 2.115617805166056
R2 Square 0.9499675901140083
__________________________________
MAE: 0.20182453851771398
MSE: 0.11531614542013391
RMSE: 0.3395823102285128
R2 Square 0.998731850588944
__________________________________


In [103]:
predicted_train = GRD_Boost2.predict(X_train_imput_scale)
predicted_test = GRD_Boost2.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.437821257809421
MSE: 4.785634599409656
RMSE: 2.187609334275582
R2 Square 0.9465045887435399
__________________________________
MAE: 0.3440713439754574
MSE: 0.2787817942477745
RMSE: 0.5279979112153518
R2 Square 0.9969341936733975
__________________________________


#### XGBoost regression model

In [104]:
xgbr = xgb.XGBRegressor(objective='reg:squarederror',  learning_rate= 0.04, max_depth=5,
                       n_estimators= 200, n_jobs=-1 )
xgbr.fit(X_train_imput_scale, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.04, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [105]:
predicted_train = xgbr.predict(X_train_imput_scale)
predicted_test = xgbr.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.550712274284493
MSE: 5.302819178079341
RMSE: 2.302785091596552
R2 Square 0.9407233279396234
__________________________________
MAE: 0.9048584349230758
MSE: 1.6694693804189715
RMSE: 1.2920794791416554
R2 Square 0.9816405881081007
__________________________________


### 7. Mix of model, which model to select for ensemble model

In [106]:
alg= [RandomForestRegressor(), AdaBoostRegressor(), GradientBoostingRegressor(), 
      DecisionTreeRegressor(), LinearRegression(), SVR(), LassoCV()]

In [109]:
for i in alg:
    model= i
    model.fit(X_train_imput_scale, y_train)
    y_pred1= model.predict(X_test_imput_scale)
    y_pred2= model.predict(X_train_imput_scale)
    print(i, model_eval(y_train, y_pred2))
    print("*"*50)
    print(i, model_eval(y_test, y_pred1))
    print("="*70)

C:\Users\HP\AppData\Local\Temp/ipykernel_22264/2741472670.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_imput_scale, y_train)


MAE: 0.441908198121265
MSE: 0.5223798748932539
RMSE: 0.7227585176898671
R2 Square 0.9942553080639327
__________________________________
RandomForestRegressor() None
**************************************************
MAE: 1.384453924914674
MSE: 4.464362569965863
RMSE: 2.112903824116437
R2 Square 0.9500958740753565
__________________________________
RandomForestRegressor() None


C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MAE: 2.3624830465841575
MSE: 8.621830338386053
RMSE: 2.9362953424998057
R2 Square 0.905184403918341
__________________________________
AdaBoostRegressor() None
**************************************************
MAE: 2.430512118310302
MSE: 10.00589252098758
RMSE: 3.1632092123328768
R2 Square 0.8881508137992404
__________________________________
AdaBoostRegressor() None


C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MAE: 1.3112293301651574
MSE: 3.3593379703055275
RMSE: 1.8328496856822514
R2 Square 0.9630568429679986
__________________________________
GradientBoostingRegressor() None
**************************************************
MAE: 1.7871826359789762
MSE: 6.533460488270553
RMSE: 2.5560634750081133
R2 Square 0.9269668110910557
__________________________________
GradientBoostingRegressor() None
MAE: 0.0
MSE: 0.0
RMSE: 0.0
R2 Square 1.0
__________________________________
DecisionTreeRegressor() None
**************************************************
MAE: 2.1778156996587037
MSE: 10.203856655290105
RMSE: 3.194347610278209
R2 Square 0.8859379050285103
__________________________________
DecisionTreeRegressor() None
MAE: 2.42219185750846
MSE: 10.749188467196358
RMSE: 3.2785955022229194
R2 Square 0.881789518940812
__________________________________
LinearRegression() None
**************************************************
MAE: 2.5268663256452206
MSE: 12.29678872564284
RMSE: 3.506677733360002
R2 Squar

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MAE: 2.1009407198120638
MSE: 10.180935030726625
RMSE: 3.190757751808593
R2 Square 0.8880386894985354
__________________________________
SVR() None
**************************************************
MAE: 2.3843115506866788
MSE: 11.871431754915484
RMSE: 3.4454944137112578
R2 Square 0.8672971973224736
__________________________________
SVR() None


C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MAE: 2.434873450872242
MSE: 10.798101969715463
RMSE: 3.2860465562306724
R2 Square 0.8812516096204276
__________________________________
LassoCV() None
**************************************************
MAE: 2.5459128414296908
MSE: 12.355273751021125
RMSE: 3.515006934704557
R2 Square 0.8618886509683472
__________________________________
LassoCV() None


### 7. Ensemble learning model  

In [110]:
# Train classifiers
reg1 = GradientBoostingRegressor(random_state=47)
reg2 = RandomForestRegressor(random_state=47)
reg3 = LinearRegression()

reg1.fit( X_train_imput_scale , y_train)
reg2.fit( X_train_imput_scale , y_train)
reg3.fit( X_train_imput_scale , y_train)

voting = VotingRegressor([("gb", reg1), ("rf", reg2), ("lr", reg3)])
voting.fit( X_train_imput_scale , y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\HP\AppData\Local\Temp/ipykernel_22264/3148182653.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg2.fit( X_train_imput_scale , y_train)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


VotingRegressor(estimators=[('gb', GradientBoostingRegressor(random_state=47)),
                            ('rf', RandomForestRegressor(random_state=47)),
                            ('lr', LinearRegression())])

In [111]:
predicted_train = voting.predict(X_train_imput_scale)
predicted_test = voting.predict(X_test_imput_scale)
model_eval(y_test, predicted_test)
model_eval(y_train, predicted_train)

MAE: 1.6994047124747513
MSE: 6.26328437571216
RMSE: 2.5026554648437247
R2 Square 0.9299869293121096
__________________________________
MAE: 1.2570371988771734
MSE: 3.1402730668491654
RMSE: 1.7720815632608917
R2 Square 0.9654659334495534
__________________________________


In [ ]:
#voting.get_params()

### Selection of the best model

In [112]:
models_liste = [lin_reg, lin_reg2, ridge_reg, Elastic_net, model_dt, rf_reg1, rf_reg2, rf_reg3, rf_reg4,
               grdB, GRD_Boost,GRD_Boost2, xgbr, voting]

In [ ]:
#GRD_Boost2

In [113]:
for model in models_liste[0:3]:
    model=model
    y_pred1= model.predict(X_test)
    y_pred2= model.predict(X_train)
    print(model)
    print('EVALUATION FOR TRAINING DATA SET')
    print(model_eval(y_train, y_pred2))
    print("*"*50)
    print('EVALUATION FOR TESTING DATA SET')    
    print(model_eval(y_test, y_pred1))
    print("="*70)
for model in models_liste[3:]:
    model=model
    y_pred1= model.predict(X_test_imput_scale)
    y_pred2= model.predict(X_train_imput_scale)
    print(model)
    print('EVALUATION FOR TRAINING DATA SET')
    print(model_eval(y_train, y_pred2))
    print("*"*50)
    print('EVALUATION FOR TESTING DATA SET')    
    print(model_eval(y_test, y_pred1))
    print("="*70)

Pipeline(steps=[('iterativeimputer', IterativeImputer()),
                ('standardscaler', StandardScaler()),
                ('selectkbest',
                 SelectKBest(k=19,
                             score_func=<function f_regression at 0x0000025E60390C10>)),
                ('linearregression', LinearRegression())])
EVALUATION FOR TRAINING DATA SET
MAE: 2.422440641833822
MSE: 10.751017484476902
RMSE: 3.278874423407658
R2 Square 0.8817694049560904
__________________________________
None
**************************************************
EVALUATION FOR TESTING DATA SET
MAE: 2.465613032612806
MSE: 11.791841090808283
RMSE: 3.4339250269637924
R2 Square 0.8681868881712305
__________________________________
None
Pipeline(steps=[('iterativeimputer', IterativeImputer()),
                ('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=19)),
                ('linearregression', LinearRegression())])
EVALUATION FOR TRAINING DATA SET
MAE: 2.42219185750846
MSE

In [227]:
def model_eval_dict(true, predicted, model_list, MAE, MSE, RMSE, R2):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    MAE.append(mae)
    MSE.append(mse)
    RMSE.append(rmse)
    R2.append(r2_square)
    return MAE, MSE, RMSE, R2

In [228]:
# evaluation  on train dataset

In [229]:
models_liste = [lin_reg, lin_reg2, ridge_reg, Elastic_net, model_dt, rf_reg1, rf_reg2, rf_reg3, rf_reg4,
               grdB, GRD_Boost,GRD_Boost2, xgbr, voting]

MAE = []
MSE = []
RMSE = []
R2 = []

for model in models_liste:
    if model in  models_liste[0:3]:
        y_pred2= model.predict(X_train)
        model_eval_dict(y_train, y_pred2, models_liste, MAE, MSE, RMSE, R2)   
    if model in models_liste[3:]:
        y_pred2= model.predict(X_train_imput_scale)
        model_eval_dict(y_train, y_pred2, models_liste, MAE, MSE, RMSE, R2)

In [230]:
# compute the dataframe with the value
models = ['linear_reg', 'linear_reg2', 'ridge_reg', 'Elastic_net', 'decision_tree', 'random_forest_reg1','random_forest_reg2', 'random_forest_reg3', 'random_forest_reg4',
               'gradien_boost_1','gradien_boost_2', 'gradien_boost_3', 'XGBRegressor', 'voting']
zipped = list(zip(models,models_liste,MAE, MSE, RMSE, R2))
df_r1 = pd.DataFrame(zipped, columns=['model','model_definition','MAE_tr', 'MSE_tr', 'RMSE_tr', 'R2_tr'])

In [231]:
df_r1

,model,model_definition,MAE_tr,MSE_tr,RMSE_tr,R2_tr
0,linear_reg,"(IterativeImputer(), StandardScaler(), SelectK...",2.422441,10.751017,3.278874,0.881769
1,linear_reg2,"(IterativeImputer(), StandardScaler(), PCA(n_c...",2.422192,10.749188,3.278596,0.881790
2,ridge_reg,"(IterativeImputer(), StandardScaler(), Ridge(a...",2.423032,10.750125,3.278738,0.881779
3,Elastic_net,"ElasticNet(alpha=0.0001, l1_ratio=0.4)",2.422465,10.749306,3.278613,0.881788
4,decision_tree,"DecisionTreeRegressor(max_depth=4, max_feature...",3.553712,23.350479,4.832233,0.743211
5,random_forest_reg1,"(DecisionTreeRegressor(max_depth=4, max_featur...",1.955381,7.327614,2.706957,0.919417
6,random_forest_reg2,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.777499,1.267010,1.125615,0.986066
7,random_forest_reg3,"(DecisionTreeRegressor(max_depth=7, max_featur...",1.331831,3.436164,1.853689,0.962212
8,random_forest_reg4,"(DecisionTreeRegressor(max_depth=8, max_featur...",1.027459,2.166910,1.472043,0.976170
9,gradien_boost_1,([DecisionTreeRegressor(criterion='friedman_ms...,1.186482,2.750446,1.658447,0.969753


In [232]:
# evaluation  on test dataset

In [233]:
models_liste = [lin_reg, lin_reg2, ridge_reg, Elastic_net, model_dt, rf_reg1, rf_reg2, rf_reg3, rf_reg4,
               grdB, GRD_Boost,GRD_Boost2, xgbr, voting]

MAE = []
MSE = []
RMSE = []
R2 = []

for model in models_liste:
    if model in  models_liste[0:3]:
        y_pred1= model.predict(X_test)
        model_eval_dict(y_test, y_pred1, models_liste, MAE, MSE, RMSE, R2)
    
    if model in models_liste[3:]:
        y_pred1= model.predict(X_test_imput_scale)
        model_eval_dict(y_test, y_pred1, models_liste, MAE, MSE, RMSE, R2)

In [234]:
zipped = list(zip(MAE, MSE, RMSE, R2))
df_r2 = pd.DataFrame(zipped, columns=['MAE_tt', 'MSE_tt', 'RMSE_tt', 'R2_tt'])

In [235]:
df_r2

,MAE_tt,MSE_tt,RMSE_tt,R2_tt
0,2.465613,11.791841,3.433925,0.868187
1,2.464620,11.782041,3.432498,0.868296
2,2.466508,11.789202,3.433541,0.868216
3,2.527915,12.300044,3.507142,0.862506
4,3.871217,28.431838,5.332151,0.682179
5,2.101229,8.956320,2.992711,0.899883
6,1.462394,4.929916,2.220341,0.944892
7,1.698172,6.205930,2.491170,0.930628
8,1.516632,5.209269,2.282382,0.941769
9,1.774146,6.391744,2.528190,0.928551


In [236]:
# concat the two dataframe (test and train score of all metric)
results = pd.concat([df_r1, df_r2], axis=1)
results

,model,model_definition,MAE_tr,MSE_tr,RMSE_tr,R2_tr,MAE_tt,MSE_tt,RMSE_tt,R2_tt
0,linear_reg,"(IterativeImputer(), StandardScaler(), SelectK...",2.422441,10.751017,3.278874,0.881769,2.465613,11.791841,3.433925,0.868187
1,linear_reg2,"(IterativeImputer(), StandardScaler(), PCA(n_c...",2.422192,10.749188,3.278596,0.881790,2.464620,11.782041,3.432498,0.868296
2,ridge_reg,"(IterativeImputer(), StandardScaler(), Ridge(a...",2.423032,10.750125,3.278738,0.881779,2.466508,11.789202,3.433541,0.868216
3,Elastic_net,"ElasticNet(alpha=0.0001, l1_ratio=0.4)",2.422465,10.749306,3.278613,0.881788,2.527915,12.300044,3.507142,0.862506
4,decision_tree,"DecisionTreeRegressor(max_depth=4, max_feature...",3.553712,23.350479,4.832233,0.743211,3.871217,28.431838,5.332151,0.682179
5,random_forest_reg1,"(DecisionTreeRegressor(max_depth=4, max_featur...",1.955381,7.327614,2.706957,0.919417,2.101229,8.956320,2.992711,0.899883
6,random_forest_reg2,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.777499,1.267010,1.125615,0.986066,1.462394,4.929916,2.220341,0.944892
7,random_forest_reg3,"(DecisionTreeRegressor(max_depth=7, max_featur...",1.331831,3.436164,1.853689,0.962212,1.698172,6.205930,2.491170,0.930628
8,random_forest_reg4,"(DecisionTreeRegressor(max_depth=8, max_featur...",1.027459,2.166910,1.472043,0.976170,1.516632,5.209269,2.282382,0.941769
9,gradien_boost_1,([DecisionTreeRegressor(criterion='friedman_ms...,1.186482,2.750446,1.658447,0.969753,1.774146,6.391744,2.528190,0.928551


In [237]:
#data_Zero_NAN # Raw data -- replace 0 with NaN---- delete 10 row 
datapath = '../data'
save_file(results, 'metrics_results_models.csv', datapath)

Writing file.  "../data\metrics_results_models.csv"
